In [1]:
import os
from cleaning import DataCleaner
from preparation import DataPrepare
from modelling import Model
import numpy as np
import pandas as pd
curr_dir = os.path.abspath('')

In [ ]:
batch_name = 'train_1'

etl_train = DataCleaner(curr_dir, 'zomato.csv')
etl_train.treat_columns()
etl_train.treat_rows()
etl_train.treat_dtypes()
etl_train.treat_rating()
etl_train.save_data(batch_name + "_cleaned.pkl")

prep = DataPrepare(curr_dir, batch_name + '_cleaned.pkl')
prep.get_binary_features()
prep.get_one_hot_features()
prep.treat_col_cuisines()
prep.save_data(batch_name + "_prepared.pkl")

train_model = Model(curr_dir, 'train')
train_model.set_data(batch_name + "_prepared.pkl")
train_model.set_folds()
summary = train_model.train('lightgbm')
train_model.save_model(batch_name + "_model_lightgbm.pkl")
print(summary)

In [2]:
batch_name = 'test_1'

etl_pred = DataCleaner(curr_dir, 'test_set_1.csv')
etl_pred.treat_columns()
etl_pred.treat_rows()
etl_pred.treat_dtypes()
etl_pred.save_data(batch_name + "_cleaned.pkl")

prep = DataPrepare(curr_dir, batch_name + '_cleaned.pkl')
prep.get_binary_features()
prep.get_one_hot_features()
prep.treat_col_cuisines()
prep.save_data(batch_name + "_prepared.pkl")

pred_model = Model(curr_dir, 'predict')
pred_model.set_data(batch_name + '_prepared.pkl')
pred_model.load_model('train_1_model_lightgbm.pkl')
y_pred = pred_model.get_predictions()

--- Renamed, dropped columns : (2208, 10)
--- Dropped appropriate rows : (2205, 10)
Done treating dtypes : (2205, 10)
before saving : (2205, 10)
--- Saved data to local ---
Dealing with nan value
Dealing with nan value
Dealing with nan value
Dealing with nan value
--- Saved data to local ---
--- Model loaded ---


In [3]:
final_res = np.concatenate([pred_model.maindf['record_id'].values.reshape(-1,1), y_pred.reshape(-1,1)], axis=1)
final_res_df = pd.DataFrame(final_res, columns=['record_id', 'predicted_rating'])
final_res_df['record_id'] = final_res_df['record_id'].astype(int)

In [4]:
final_res_df.head()

,record_id,predicted_rating
0,0,3.244274
1,1,3.389014
2,2,3.206916
3,3,3.097213
4,4,3.243915
